In [76]:
import os
import mlflow
import numpy as np
import pandas as pd

from math import sqrt
from joblib import delayed
from joblib import Parallel

from warnings import catch_warnings
from warnings import filterwarnings
from multiprocessing import cpu_count
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.statespace.sarimax import SARIMAX

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from pathlib import Path
from functools import reduce
from datetime import datetime

from prophet import Prophet

# Data

In [79]:
df = pd.read_csv('./../final_dfs/for_modeling/df_final_date_wide_2007.csv', converters={'ags2': str, 'ags5': str} )
df.set_index('ags5', drop=True, inplace=True)
df = df.drop("Unnamed: 0", axis=1)
df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d')
df = df[(df['date'] >= '2010-01-01') & (df['date'] < '2020-01-01')]

In [80]:
# select only column for which all values are avaliable
df = df[['date', 
        'number_of_company_deletions', 
        'number_of_company_liquidations', 
        'number_of_start_ups',
        'unemployment_rate', 
        'registerd_jobs',
        'employees_social_security_at_work',
        'employees_social_security_at_residence']]

In [81]:
# rename date and unemployment_rate for Prophet function 
df = df.rename(columns={'date': 'ds', 'unemployment_rate': 'y'})

# 401 Model for each kreis

In [99]:
# one-step forecast
def Prophet_forecast(data, config):
    g, sps, ys, sm = config
    # define model
    model = Prophet(growth = g, seasonality_prior_scale = sps, yearly_seasonality = ys, seasonality_mode = sm)
    # fit model
    model_fit = model.fit(data)
    # make one step forecast
    future = model_fit.make_future_dataframe(periods=12, freq='MS')
    res = model_fit.predict(future)
    res.to_csv(f"runs_res/{g}_{sps}_{ys}_{sm}")
    return res.iloc[-12:][["ds","yhat"]]

In [93]:
df_ags5 = df.filter(regex='1001', axis=0)
config = ['linear', 15.0, True, 'additive']
Prophet_forecast(df_ags5, config)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,ds,yhat
108,2019-01-01,8.630515
109,2019-02-01,8.582007
110,2019-03-01,8.462736
111,2019-04-01,8.165552
112,2019-05-01,7.952423
113,2019-06-01,7.719569
114,2019-07-01,7.914310
115,2019-08-01,8.018994
116,2019-09-01,7.612764
117,2019-10-01,7.535532


In [84]:
# root mean squared error or rmse
def measure_rmse(actual, predicted):
    return sqrt(mean_squared_error(actual, predicted))

In [85]:
# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
    return data[:-n_test], data[-n_test:]

In [86]:
# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
    # split dataset
    train, test = train_test_split(data, n_test)
    # fit model and make forecast for history
    yhat = Prophet_forecast(train, cfg)
    # estimate prediction error
    error = measure_rmse(test['y'], yhat['yhat'])
    return error

In [100]:
def run_model(data, n_test): 
        for g in ['linear']: 
            for sps in [10.0, 15.0]: 
                for ys in ['auto', True]: 
                    for sm in ['additive', 'multiplicative']:
                        with mlflow.start_run():
                            params = [g, sps, ys, sm]

                            ags5 = list(set(df.index.values.tolist()))
                            rmse = []
                            for i in ags5:
                                df_ags5 = df.filter(regex=i, axis=0)
                                current_rmse = walk_forward_validation(df_ags5, n_test, params)
                                print(current_rmse)
                                rmse.append(current_rmse)

                            # Log params
                            params = {"growth":g, "sps":sps, "ys":ys, "sm":sm}

                            mlflow.log_params(params)

                            # Log metrics
                            mlflow.log_metric('rmse', np.mean(rmse))
                            mlflow.log_metric('mse', np.mean(rmse)**2)

In [ ]:
run_model(df, 12)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


0.28422807679277473


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


0.24569282055257768


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

0.18984383428209423


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


0.14448621202761236


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


0.43018557481716163


In [20]:
np.mean([10.0, 15.0])

12.5

In [9]:
# score a model, return None on failure
def score_model(data, n_test, cfg, debug=False):
    result = None
    # convert config to a key
    key = str(cfg)
    # show all warnings and fail on exception if debugging
    if debug:
        result = walk_forward_validation(data, n_test, cfg)
    else:
            # one failure during model validation suggests an unstable config
        try:
            # never show warnings when grid searching, too noisy
            with catch_warnings():
                filterwarnings("ignore")
                result = walk_forward_validation(data, n_test, cfg)
        except:
            error = None
    # check for an interesting result
#     if result is not None:
#         print(' > Model[%s] %.3f' % (key, result))
        
    print('.', end='')
    return (key, result)

----------
!mlflow experiments create --experiment-name 401ModelsTemp

Created experiment '401ModelsTemp' with id 6

---------

In [ ]:
# data split
n_test = 12
# model config
cfg = [('linear'), (10.0)]
# train 401 models
print("Training", end=' ')
failed_ags5 = []
rmse_scores = []

with mlflow.start_run(experiment_id=6):
    
    for ags5, df_for_kreis in df_y_grouped:
        
        data = list(df_for_kreis.sort_values('ds').drop(['ds'], axis=1).copy()['y'])
        
        try: 
        
            score = score_model(data, n_test, cfg)

            if score is None:
                raise Exception("No Score")
            
            rmse_scores.append(score[1])
            print('.', end='')
        except:
            failed_ags5.append(ags5)
            print('!', end='')
            
    params = {}
    params['p'], params['d'], params['q'] = cfg[0]
    params['s_p'], params['s_d'], params['s_q'], params['m'] = cfg[1]
    params['t'] = cfg[2]
    
    net_rmse = np.mean([rmse**2 for rmse in rmse_scores])**(0.5)
    
    
    mlflow.log_params(params)
    mlflow.log_metric('net_rmse', net_rmse)
      
    

print('\ndone')
print("Failed ags5:", failed_ags5)

Training ........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [ ]:
rmse_scores

In [ ]:
params

In [37]:
m = Prophet()
m.fit(df)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [38]:
future = m.make_future_dataframe(periods=365)
future.tail()

,ds
480,2020-11-26
481,2020-11-27
482,2020-11-28
483,2020-11-29
484,2020-11-30


In [42]:
Prophet?